# Partial wave decomposition of One-Pion Exchange Diagram

See also https://arxiv.org/src/1911.04877v2/anc/supplemental.pdf

## Define matrix element

In [ ]:
from sympy import symbols, solve, sympify, Symbol
from sympy.physics.vector import ReferenceFrame
import numpy as np

In [ ]:
eqs = (
    "p12i - p1i + p2i",
    "p12o - p1o + p2o",
    "P12i - p1i - p2i - k",
    "P12o - p1o - p2o",
)
mc = {"P12o": "P12i"}
subs = solve(eqs, "p1i", "p2i", "p1o", "p2o")

mom_subs = {
    "q1": sympify("p1o - p1i").subs(subs).subs(mc),
    "q2": sympify("p2o - p2i").subs(subs).subs(mc),
}

print(mom_subs)

In [ ]:
m_pi = symbols("m_pi")

In [ ]:
N = ReferenceFrame("N")

sigma1 = np.dot(symbols("sigma11 sigma12 sigma13"), [N.x, N.y, N.z])
sigma2 = np.dot(symbols("sigma21 sigma22 sigma23"), [N.x, N.y, N.z])

q = Symbol("q") * N.z
p12i = np.dot(symbols("p_i1 p_i2 p_i3"), [N.x, N.y, N.z])
p12o = np.dot(symbols("p_o1 p_o2 p_o3"), [N.x, N.y, N.z])

q1 = q / 2 - p12i / 2 + p12o / 2
q2 = q / 2 + p12i / 2 - p12o / 2

In [ ]:
fact = sympify("3 * g_A ** 2 / 16 / f_pi ** 2 / m_N")

In [ ]:
ope1 = sigma1.dot(q) * sigma2.dot(q2) / (q2.dot(q2) + m_pi ** 2)
ope1 += ope1.subs({"q21": "q11", "q22": "q12", "q23": "q13"})
ope1 *= fact
ope1

In [ ]:
ope2 = sigma1.dot(q2) * sigma2.dot(q2) * q2.dot(q) / (q2.dot(q2) + m_pi ** 2) ** 2
ope2 += ope2.subs({"q21": "q11", "q22": "q12", "q23": "q13"})
ope2 *= fact
ope2

## Partial wave decomposition

In [ ]:
from numpwd.integrate.analytic import SPHERICAL_BASE_SUBS, ANGLE_BASE_SUBS
from numpwd.operators.expression import decompose_operator
from yaml import dump

In [ ]:
HBARC = 197.327
constants = dict(m_N=938.918 / HBARC, g_A=1.29, m_pi=138.03 / HBARC, f_pi=92.4 / HBARC)

In [ ]:
import os
from numpwd.densities.h5 import read_h5
import re

DATA = os.environ["DATA"]
DENS_DIR = os.path.join(DATA, "4he", "chsms")
DENSITIES = [
    el
    for el in os.listdir(DENS_DIR)
    if el.endswith(".h5") and "th=1.80E+02" in el and "cut=4" in el
]

f = DENSITIES[0]
dens = read_h5(os.path.join(DENS_DIR, f))

pattern = r"\-om=([0-9\.]+E[\+\-][0-9]+)"
omega = np.sort([float(re.search(pattern, f).group(1)) for f in DENSITIES])
q = omega / HBARC * 2

In [ ]:
ope_expr1 = ope1.subs(SPHERICAL_BASE_SUBS).subs(ANGLE_BASE_SUBS).simplify()

In [ ]:
isospin_expression = sympify("tau10 * tau20")
args = [["p_o", dens.p.tolist()], ["p_i", dens.p.tolist()], ["q", q.tolist()]]
substitutions = [
    SPHERICAL_BASE_SUBS,
    ANGLE_BASE_SUBS,
    constants,
]
spin_decomposition_kwargs = {}
integration_kwargs = {
    "nx": 10,
    "nphi": 16,
    "lmax": 5,
    "numeric_zero": 1.0e-8,
    "m_lambda_max": None,
}

In [ ]:
ope1_kwargs = dict(
    spin_momentum_expression=str(ope1),
    isospin_expression=str(isospin_expression),
    args=args,
    substitutions=substitutions,
    spin_decomposition_kwargs=spin_decomposition_kwargs,
    integration_kwargs=integration_kwargs,
    misc={
        "op-name": "one-pion exchange 1",
        "momentum-mesh": {
            "meshtype": "TRNS",
            "n1": 28,
            "n2": 8,
            "ntot": 36,
            "p1": 1.0,
            "p2": 7.0,
            "p3": 20.0,
        },
    },
)

with open("ope1.yml", "w") as out:
    dump(ope1_kwargs, out)

In [ ]:
ope2_kwargs = ope1_kwargs.copy()
ope2_kwargs["spin_momentum_expression"] = str(ope2)

with open("ope2.yml", "w") as out:
    dump(ope2_kwargs, out)

In [ ]:
op_ope1 = decompose_operator(**ope1_kwargs)